Install pustaka yang diperlukan


In [ ]:
! pip install datasets transformers rouge-score nltk

Login ke akun huggingface

In [ ]:
!huggingface-cli login

install dan setup git-lfs

In [ ]:
!apt install git-lfs
!git config --global user.email "arkanfadhil080@gmail.com.com"
!git config --global user.name "Muhammad Fadhil Arkan"

download tokenzer punkt dari library nltk

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Fine-tuning pada model untuk melakukan tugas generasi pertanyaan

In [5]:
model_checkpoint = "t5-small"

## Loading the dataset

download dataset dan metric yang akan digunakan

In [ ]:
from datasets import load_dataset, load_metric

#raw_datasets = load_dataset("xsum")
raw_datasets = load_dataset("squad")

metric = load_metric("rouge")

melihat dataset

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

melihat isi dataset yang sebenarnya

In [8]:
raw_datasets["train"][0]

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

visualisasi dataset

In [9]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=20):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(raw_datasets["train"])

,id,title,context,question,answers
0,572780475951b619008f8b7c,Alsace,"Holy Roman Empire central power had begun to decline following years of imperial adventures in Italian lands, often ceding hegemony in Western Europe to France, which had long since centralized power. France began an aggressive policy of expanding eastward, first to the rivers Rhône and Meuse, and when those borders were reached, aiming for the Rhine. In 1299, the French proposed a marriage alliance between Philip IV of France's sister Blanche and Albert I of Germany's son Rudolf, with Alsace to be the dowry; however, the deal never came off. In 1307, the town of Belfort was first chartered by the Counts of Montbéliard. During the next century, France was to be militarily shattered by the Hundred Years' War, which prevented for a time any further tendencies in this direction. After the conclusion of the war, France was again free to pursue its desire to reach the Rhine and in 1444 a French army appeared in Lorraine and Alsace. It took up winter quarters, demanded the submission of Metz and Strasbourg and launched an attack on Basel.",The French proposed a marriage between which two people?,"{'text': ['Blanche and Albert'], 'answer_start': [440]}"
1,570c2ab3ec8fbc190045bcfc,Federal_Bureau_of_Investigation,"In 2000, the FBI began the Trilogy project to upgrade its outdated information technology (IT) infrastructure. This project, originally scheduled to take three years and cost around $380 million, ended up going far over budget and behind schedule. Efforts to deploy modern computers and networking equipment were generally successful, but attempts to develop new investigation software, outsourced to Science Applications International Corporation (SAIC), were not. Virtual Case File, or VCF, as the software was known, was plagued by poorly defined goals, and repeated changes in management. In January 2005, more than two years after the software was originally planned for completion, the FBI officially abandoned the project. At least $100 million (and much more by some estimates) was spent on the project, which never became operational. The FBI has been forced to continue using its decade-old Automated Case Support system, which IT experts consider woefully inadequate. In March 2005, the FBI announced it was beginning a new, more ambitious software project, code-named Sentinel, which they expected to complete by 2009.",When did the FBI begin the Trilogy project?,"{'text': ['2000'], 'answer_start': [3]}"
2,56d8df98dc89441400fdb36e,2008_Summer_Olympics_torch_relay,"Of the 80 torch-bearers in London, Sir Steve Redgrave, who started the relay, mentioned to the media that he had received e-mailed pleas to boycott the event and could ""see why they would like to make an issue"" of it. Francesca Martinez and Richard Vaughan refused to carry the torch, while Konnie Huq decided to carry it and also speak out against China. The pro-Tibetan Member of Parliament Norman Baker asked all bearers to reconsider. Amid pressure from both directions, Prime Minister Gordon Brown welcomed the torch outside 10 Downing Street without holding or touching it. The London relay saw the torch surrounded by what the BBC described as ""a mobile protective ring."" Protests began as soon as Redgrave started the event, leading to at least thirty-five arrests. In Ladbroke Grove a demonstrator attempted to snatch the torch from Konnie Huq in a momentary struggle, and in a separate incident, a fire extinguisher was set off near the torch. The Chinese ambassador carried the torch through Chinatown after an unpublicized change to the route amid security concerns. The torch made an unscheduled move onto a bus along Fleet Street amid security concerns and efforts to evade the protesters. In an effort to counter the pro-Tibet protesters and show their support for the 2008 Beijing Olympics, more than 2,000 Chinese also gathered on the torch route and demonstrated with signs, banners and Chinese flags. A 

informasi tentang metric yang digunakan

In [ ]:
metric

cara menggunakan metric

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
aaa=metric.compute(predictions=fake_preds, references=fake_labels)
aaa

## Preprocessing the data

download tokenizer


In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tambahkan prefix pada input 

In [14]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

preprocess dataset

In [15]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["context"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["question"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

aplikasikan preprocessing pada seluruh dataset

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

## Fine-tuning the model

Download model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
model

inisialisasi argumen untuk training

In [19]:
batch_size = 10
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    "test-summarization-2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-xsum-3",
)

gunakan data collator untuk melakukan padding pada input

In [20]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

definisikan fungsi untuk menghitung metric

In [21]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

Inisialisasi trainer

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

mulai proses training


In [ ]:
trainer.train()

upload hasil training ke huggingface hub

In [ ]:
trainer.push_to_hub()